In [15]:
import timm
import torch
import torchvision.transforms as T
import numpy as np
from PIL import Image
from sklearn.metrics.pairwise import cosine_similarity
import gdown
import os
import pickle

# Load the pre-trained animal re-identification model
model = timm.create_model("hf-hub:BVRA/MegaDescriptor-S-224", pretrained=True)
model = model.eval()  # Set the model to evaluation mode

# Define the transformation pipeline for input images
train_transforms = T.Compose([
    T.Resize((224, 224)), 
    T.ToTensor(), 
    T.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])

# Function to extract features from an image file path
def extract_features(image_path):
    img = Image.open(image_path)  # Load image from file path
    img_transformed = train_transforms(img)  # Apply transformations
    img_features = model(img_transformed.unsqueeze(0))  # Extract features
    return img_features.detach().numpy()  # Convert to numpy array

# Dummy known animal feature list and information list
known_animal_features = np.load('Feature CSV/known_animal_features.npy')  # Initially empty array for features
#animal_info_list = []  # Initially empty list for animal info

# Load the list from the file
with open('Feature CSV/animal_info_list.pkl', 'rb') as f:
    animal_info_list = pickle.load(f)

# Function to download image from Google Drive
def download_image_from_drive(drive_url):
    # Convert Google Drive link to direct download link
    #https://drive.google.com/open?id=1KJ6kpya6UzhxteN-kIhg6mITvk_ekBnj
    file_id = drive_url.split('=')[1]
    direct_link = f'https://drive.google.com/uc?id={file_id}'
    output_path = f'Test Images/Registered Cows/{file_id}.jpg'  # Save as a JPEG file (or appropriate format)
    gdown.download(direct_link, output_path, quiet=False)
    return output_path

# Function to add a new animal to the known list
def add_animal(drive_links, name, owner, refnum):
    # Download images and extract features
    features_list = []
    for link in drive_links:
        image_path = download_image_from_drive(link)  # Download image
        features_list.append(extract_features(image_path))  # Extract features
        
    avg_features = np.mean(features_list, axis=0)  # Average the features
    
    # Update the known features list
    global known_animal_features
    if known_animal_features.size == 0:
        known_animal_features = avg_features
    else:
        known_animal_features = np.vstack((known_animal_features, avg_features))
    
    # Update the animal information list
    animal_info_list.append({
        'name': name,
        'owner': owner,
        'refnum' : refnum
    })
    print(f"Added animal: {name}, Owner: {owner}, Reference Number: {refnum}")

In [16]:
#Write Python Code to Access Google Sheets
#Import Libraries and Set Up Authentication:
import gspread
from oauth2client.service_account import ServiceAccountCredentials

#Set Up the Google Sheets API Client:

# Define the scope
scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']

# Load the service account credentials
creds = ServiceAccountCredentials.from_json_keyfile_name('D:/5 th Semester/Image Processing and Machine Vision/Project/Google Cloud Console/animal-re-identification-ae729ae62110.json', scope)

# Authorize the client
client = gspread.authorize(creds)

#Open the Google Sheet:
# Open the sheet by name or by URL
sheet = client.open("Cow Re-Identification").sheet1  # Use .sheet1 to access the first sheet

# Define the column to search (e.g., Column A corresponds to index 1)
column_index = 2  # Adjust for the column you want to search
search_value = input("Enter your reference number: ")  # Replace with the content you're searching for

# Get all values in the specified column
column_values = sheet.col_values(column_index)

# Search for the value and get the row index
try:
    row_index = column_values.index(search_value) + 1  # Adding 1 because list indices start at 0
    print(f"Found '{search_value}' at row: {row_index}")
except ValueError:
    print(f"'{search_value}' not found in column {column_index}.")



cell_value = sheet.cell(row_index, 6).value
# Split the string into a list using the comma as a separator
new_animal_links = cell_value.split(", ")

name = sheet.cell(row_index, 5).value
owner = sheet.cell(row_index, 3).value
ref = sheet.cell(row_index, 2).value

Found 'REF20240925234222' at row: 10


In [17]:
add_animal(new_animal_links, name, owner, ref)

Downloading...
From: https://drive.google.com/uc?id=1k9BIvwwuj7ooHji5HJzVfv0Hs4rzQWP9
To: d:\5 th Semester\Image Processing and Machine Vision\Project\VSCode\Test Images\Registered Cows\1k9BIvwwuj7ooHji5HJzVfv0Hs4rzQWP9.jpg
100%|██████████| 56.7k/56.7k [00:00<00:00, 335kB/s]


Added animal: Mashi, Owner: Ravindu, Reference Number: REF20240925234222


In [18]:
# Save the array to a file
np.save('Feature CSV/known_animal_features.npy', known_animal_features)

# Save the list to a file
with open('Feature CSV/animal_info_list.pkl', 'wb') as f:
    pickle.dump(animal_info_list, f)

print('Your Cow Regisrtation is Sucessfull...')

Your Cow Regisrtation is Sucessfull...


In [19]:
animal_info_list

[{'name': 'Konda Suddi', 'owner': 'Rajitha Niroshan'},
 {'name': 'Konda Suddi',
  'owner': 'Rajitha Niroshan',
  'refnum': 'REF20240921053116'},
 {'name': 'Mashi', 'owner': 'Ravindu', 'refnum': 'REF20240925234222'}]